In [2]:

from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from utils import ProgramDataset, make_max_length
from generate import *



In [3]:
from collections.abc import Iterable, Iterator
from functools import reduce
def canBeList(x):
    return isinstance(x, Iterable)

def expand_iters(e):
    if(canBeList(e)): # Expand it out.
        return [expand_iters(ele) for ele in e]
    return e
def mapt(f, x):
    if(len(x) == 0):
        return f(x)
    # TODO: this
    return x
def cons(a, b):
    blist = canBeList(b)
    alist = canBeList(a)
    al = expand_iters(a) if alist else [a]
    bl = expand_iters(b) if blist else [b]
    return al + bl

foldl = lambda f, acc, xs: reduce(f, xs, acc)
foldr = lambda f, acc, xs: reduce(lambda x, y: f(y, x), xs[::-1], acc)
def recl(f, e, x):
    if(len(x) == 0):
        return e
    return f(x[0], x[1:])

def do_gpt(seed_text, tokenizer, model, temperature=0.5):
    input_tensor = tokenizer(seed_text, return_tensors="pt").input_ids.cuda()
    outputs = model.generate(
        input_tensor, 
        max_length=2048,  
        # num_return_sequences=5,
        # no_repeat_ngram_size=2,
        # repetition_penalty=1.5,
        top_p=0.95,
        temperature=temperature,
        do_sample=True,
        top_k=50,
        # early_stopping=False
    )
    output = outputs[0]
    out_str = tokenizer.decode(output) 
    progSep = "Program:"
    if progSep in out_str:
        out_str =out_str.split(progSep)[1]
    out_prog = out_str.replace("<|endoftext|>", "")
    return out_prog

In [4]:

# Now, make the outputs for us to evaluate:
fine_model, fine_tokenizer = get_model("../../../output/haskell-parsed", "125M", "haskell-parsed")


In [7]:
prompt = """
(L ({Span}) (SigD (NoExtField) (TypeSig (NoExtField) ((:) (L ({Span}) (Unqual ({OcN}f))) ([])) (HsWC (NoExtField) (HsIB (NoExtField) (L ({Span}) (HsFunTy (NoExtField) (L ({Span}) (HsTyVar (NoExtField) (NotPromoted) (L ({Span}) (Unqual ({OcN}Int))))) (L ({Span}) (HsTyVar (NoExtField) (NotPromoted) (L ({Span}) (Unqual ({OcN}Bool))))))))))))
"""
# That's f :: Int -> Bool btw
print(do_gpt(prompt + "\n", fine_tokenizer, fine_model, temperature=0.2))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


f :: Int -> Bool
fmap (L ({Span}) (FunBind (NoExtField) (L ({Span}) (Unqual ({OcN}main))) (MG (NoExtField) (L ({Span}) ((:) (L ({Span}) (Match (NoExtField) (FunRhs (L ({Span}) (Unqual ({OcN}main))) (Prefix) (NoSrcStrict)) ([]) (GRHSs (NoExtField) ((:) (L ({Span}) (GRHS (NoExtField) ([]) (L ({Span}) (HsDo (NoExtField) (DoExpr) (L ({Span}) ((:) (L ({Span}) (BindStmt (NoExtField) (L ({Span}) (ListPat (NoExtField) ((:) (L ({Span}) (VarPat (NoExtField) (L ({Span}) (Unqual ({OcN}x))))) ([])))) (L ({Span}) (HsVar (NoExtField) (L ({Span}) (Unqual ({OcN}getArgs))))) (SyntaxExpr (HsLit (NoExtField) (HsString (NoSourceText) ({abstract:FastString}))) ([]) (WpHole)) (SyntaxExpr (HsLit (NoExtField) (HsString (NoSourceText) ({abstract:FastString}))) ([]) (WpHole)))) ((:) (L ({Span}) (BodyStmt (NoExtField) (L ({Span}) (OpApp (NoExtField) (L ({Span}) (HsVar (NoExtField) (L ({Span}) (Unqual ({OcN}print))))) (L ({Span}) (HsVar (NoExtField) (L ({Span}) (Unqual ({OcN}$))))) (L ({Span}) (HsApp (NoExtField) 

In [6]:
longstr = """
Examples:
Inputs: 
[9, 14, -12, 7, 8]
Output: 
[9, 14, -12, 7, 8, 9, 14, -12, 7, 8]
Inputs: 
[-14, 3]
Output: 
[-14, 3, -14, 3]
Inputs: 
[-9, 13, -19, -4, 14]
Output: 
[-9, 13, -19, -4, 14, -9, 13, -19, -4, 14]
Inputs: 
[6, -14]
Output: 
[6, -14, 6, -14]
Inputs: 
[5, -7]
Output: 
[5, -7, 5, -7]
Inputs: 
[-14, 20, -11]
Output: 
[-14, 20, -11, -14, 20, -11]
Inputs: 
[17]
Output: 
[17, 17]

Program:
"""

# Original program: 
"""

Program: 
<prog> -> "lambda" <lamargs> ":" <stmt> ATTRS: {retType=[[int]], y_is_decl=true}
	"lambda" ->  ATTRS: {}
	<lamargs> -> <varInit> ATTRS: {length=1, y_is_decl=true}
		<varInit> -> <lowercaseAscii> ATTRS: {y_is_decl=true}
			<lowercaseAscii> -> "y" ATTRS: {chosenSymbol=y, retType=[[int]]}
				"y" ->  ATTRS: {}
	":" ->  ATTRS: {}
	<stmt> -> "cons" "(" <declared> "," <declared> ")" ATTRS: {retType=[[int]], y_is_decl=true}
		"cons" ->  ATTRS: {}
		"(" ->  ATTRS: {}
		<declared> -> <lowercaseAscii> ATTRS: {chosenSymbol=y, retType=[[int]], y_is_decl=true}
			<lowercaseAscii> -> "y" ATTRS: {chosenSymbol=y, retType=[[int]], y_is_decl=true}
				"y" ->  ATTRS: {}
		"," ->  ATTRS: {}
		<declared> -> <lowercaseAscii> ATTRS: {chosenSymbol=y, retType=[[int]], y_is_decl=true}
			<lowercaseAscii> -> "y" ATTRS: {chosenSymbol=y, retType=[[int]], y_is_decl=true}
				"y" ->  ATTRS: {}
		")" ->  ATTRS: {}
		"""

out = do_gpt(longstr)
print(out)

TypeError: do_gpt() missing 2 required positional arguments: 'tokenizer' and 'model'

In [ ]:
import re
cartp = """
Inputs: 
[[1], [2, 3]]
Outputs:
[[1, 2], [1, 3]]
Inputs: 
[[1, 2, 3], [4, 5]]
Output:
[[1, 4], [1, 5], [2, 4], [2, 5], [3, 4], [3, 5]]
Inputs: 
[[1], [2, 3], [4, 5]]
Output:
[[1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5]]

Program:
"""
func_raw_str = do_gpt(cartp)
for lin in func_raw_str.split("\n"):
    print(lin)
symbols = re.findall('"([\w\(\),:\[\]\d-]+)"', func_raw_str)
func_str = ' '.join(symbols)
print(symbols)
print(func_str)
func = eval(func_str)
print(expand_iters(func([[1, 2, 3], [4, 5]])))



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



20 {retType=[[int]]}
	"lambda"   {}
	5 {length=1, 0.retType=[[int]]}
		2 {retType=[[int]]}
			75 {retType=[[int]]}
				"z"   {}
	":"   {}
	15 {retType=[[int]]}
		"map"   {}
		"("   {}
		20 {retType=[int]}
			"lambda"   {}
			5 {length=1, 0.retType=int}
				2 {retType=int}
					63 {retType=int}
						"i"   {}
			":"   {}
			18 {retType=[int], 2.retType=[int]}
				"foldr"   {}
				"("   {}
				20 {retType=[int]}
					"lambda"   {}
					4 {0.retType=int, 1.retType=[[int]], length=2}
						5 {length=1, 0.retType=int}
							2 {retType=int}
								65 {retType=int}
									"k"   {}
						","   {}
						2 {retType=[[int]]}
							76 {retType=[[int]]}
								"a"   {}
					":"   {}
					15 {retType=[int]}
						"map"   {}
						"("   {}
						20 {retType=int}
							"lambda"   {}
							5 {length=1, 0.retType=int}
								2 {retType=int}
									63 {retType=int}
										"i"   {}
							":"   {}
							12 {retType=int, 2.retType=int, 0.retType=[int]}
								6 {retType=[int]}
									3 {

NameError: name 't' is not defined